In [1]:
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import sketch
from shapely.geometry import Point
import time
from translate import Translator

# Configurar pandas para mostrar todas las filas y columnas
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)




In [2]:
df_modificado = pd.read_excel("E:\Cursos\BC_Data_Science\Repositorio\ONLINE_DS_THEBRIDGE_V\proyecto EDA\DATASETS\DATASETS MODIFICADOS\ciber_modificado.xlsx")

### unir los excel de karspexky que solo tienen un campo country y otro de datos numeritcos, con la intecion qde al final quedarme una lista de counyry y 3 datos numericos

In [3]:
# los datos que tengo son extraidos el 30/11 por lo que se actualizan el mes que viene
# he creado un excel con todos los tipos de ataques siendo este

df= pd.read_excel(r"E:\Cursos\BC_Data_Science\Repositorio\ONLINE_DS_THEBRIDGE_V\proyecto EDA\DATASETS\KARPESKY\todo_en_uno_scan_day_2023-11-30.xlsx")

In [16]:
df.head(5)

,Tipo_Ataque,Exploits,Tipo_Ataque.1,Ransomware,Tipo_Ataque.2,Amenazas_web,Tipo_Ataque.3,Ataques_red,Tipo_Ataque.4,Bajo_demanda
0,República de Guinea Ecuatorial,0.00365,República Islámica de Afganistán,0.00100,República de Moldova,0.02808,República de Corea,0.02492,República de Guinea-Bissau,0.03080
1,República de la Unión de Myanmar,0.00156,Turkmenistán,0.00085,República de Letonia,0.02663,República Socialista de Vietnam,0.01109,Turkmenistán,0.02693
2,Estados Unidos de América,0.00128,Reino de Lesotho,0.00085,República de Malawi,0.02638,República Popular de China,0.01032,República Argelina Democrática y Popular,0.02682
3,Guayana,0.00126,República Islámica de Pakistán,0.00078,República de Madagascar,0.02608,República de Ghana,0.01014,República Islámica de Afganistán,0.02666
4,Canadá,0.00107,República Islámica de Irán,0.00078,"Nación de Brunei, Morada de la Paz",0.02577,República Islámica de Irán,0.00871,República de la Unión de Myanmar,0.02625


In [24]:
df.Tipo_Ataque.unique()

array(['República de Guinea Ecuatorial',
       'República de la Unión de Myanmar', 'Estados Unidos de América',
       'Guayana', 'Canadá', 'República de Níger', 'Principado de Andorra',
       'República Federal de Alemania', 'República de Malawi',
       'República Socialista de Vietnam', 'República Centroafricana',
       'Gran Ducado de Luxemburgo', 'Mancomunidad de Australia',
       'República Argelina Democrática y Popular',
       'Serenísima República de San Marino', 'Bailía de Jersey',
       'Principado de Liechtenstein', 'República francés', 'Islandia',
       'Japón', 'República Portuguesa', 'Reino de España',
       'Commonwealth de las Bahamas',
       'República Democrática Federal de Nepal', 'República de Austria',
       'Macao, Región Administrativa Especial de la República Popular China',
       'República de Mozambique', 'República de Guinea-Bissau',
       'Reino de Noruega', 'Estado de Qatar',
       'Reino Unido de Gran Bretaña e Irlanda del Norte',
       'Rep

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tipo_Ataque    174 non-null    object 
 1   Exploits       174 non-null    float64
 2   Tipo_Ataque.1  145 non-null    object 
 3   Ransomware     145 non-null    float64
 4   Tipo_Ataque.2  193 non-null    object 
 5   Amenazas_web   193 non-null    float64
 6   Tipo_Ataque.3  187 non-null    object 
 7   Ataques_red    187 non-null    float64
 8   Tipo_Ataque.4  193 non-null    object 
 9   Bajo_demanda   193 non-null    float64
dtypes: float64(5), object(5)
memory usage: 15.2+ KB


In [4]:
#NO HAY VALORES DIFERENTES, TODOS SON UNICOS A LAS 4 COLUNAS DE TIPO DE ATAUQE, ASI QYE VAMOS CRERAR UNA COLUMNA SOLA DE TIPO DE ATAQUE)
# Crear un conjunto de todos los valores únicos en las columnas, excluyendo NaN
valores_unicos_totales = set()
for columna in df.columns:
    valores_unicos_totales.update(set(df[columna].dropna().astype(str).unique()))

# Incluir el valor 0.00365 en el conjunto de valores únicos
valores_unicos_totales.add(str(0.00365))

# Inicializar un diccionario para almacenar a qué columnas pertenecen los valores únicos
columnas_por_valor = {valor: [] for valor in valores_unicos_totales}

# Iterar sobre las columnas y filas para asignar a qué columnas pertenecen los valores únicos
for columna in df.columns:
    for indice, valor in df[columna].items():
        # Verificar si el valor no es NaN
        if pd.notna(valor):
            columnas_por_valor[str(valor)].append(columna)

# Encontrar valores fuera de los valores únicos
valores_fuera = {columna: set(df[columna].dropna().astype(str).unique()) - valores_unicos_totales for columna in df.columns}

# Obtener el número total de valores únicos y el número total de valores fuera de los únicos
num_valores_unicos_totales = len(valores_unicos_totales)
num_valores_fuera_totales = sum(len(valores) for valores in valores_fuera.values())

# Imprimir resultados
print("Valores únicos en todas las columnas:")
print(valores_unicos_totales)
print(num_valores_unicos_totales)

print("\nA qué columnas pertenecen cada valor único:")
for valor, columnas in columnas_por_valor.items():
    print(f"{valor}: {columnas}")

print("\nValores fuera de los valores únicos y a qué columnas pertenecen:")
for columna, valores in valores_fuera.items():
    print(f"{columna}: {valores}")
print(num_valores_fuera_totales)

Valores únicos en todas las columnas:
{'0.00879', 'Estado Independiente de Papúa Nueva Guinea', '0.00365', '0.02141', '0.00866', '0.00535', 'Congo (Democratic Republic of the)', '0.00808', '0.00598', '0.0001', '0.00243', '0.01491', '0.00846', '0.01188', '0.00842', '0.00569', 'Departamento de Mayotte', '0.00124', '0.00318', '0.00784', '0.00874', '0.00275', '0.01657', 'República de Serbia', '0.00099', '0.01213', 'Federación de Rusia', '0.00076', '0.02558', '0.01588', '0.00349', '0.01064', '0.00745', 'República de Tayikistán', '0.01004', 'República de Albania', '0.00397', '0.00239', '0.00166', 'República de Túnez', '0.00027', '0.0096', 'Reino de Marruecos', '0.00219', 'República de Mozambique', '0.00016', '0.01561', '0.00935', 'República de Macedonia del Norte', '0.00341', 'República de Mauricio', '0.00596', '0.02152', '0.017', '0.02107', '0.00262', '0.0099', 'República Islámica de Mauritania', '0.0159', '0.02577', '0.02032', '0.00266', '0.01615', '0.01025', '0.01264', '0.0129', 'Estado d

In [5]:
# UNIION COLUMNAS TIPOS DE ATAQUE EN UNA SOLA RESPECTANDO SU CORRELACION CON LOS VALORES DEL RESTO DE COLUMNAS
# Unir las columnas de tipo de ataque en una sola columna
df['Tipo_Ataque_Combinado'] = df['Tipo_Ataque'].fillna(df['Tipo_Ataque.1']).fillna(df['Tipo_Ataque.2']).fillna(df['Tipo_Ataque.3']).fillna(df['Tipo_Ataque.4'])

# Imprimir el DataFrame resultante
print(len(df[['Tipo_Ataque_Combinado', 'Tipo_Ataque', 'Tipo_Ataque.1', 'Tipo_Ataque.2', 'Tipo_Ataque.3', 'Tipo_Ataque.4']]))


columna_unica = df[['Tipo_Ataque_Combinado', 'Tipo_Ataque', 'Tipo_Ataque.1', 'Tipo_Ataque.2', 'Tipo_Ataque.3', 'Tipo_Ataque.4']]





193


In [6]:
#olumna_unica.head(5)

In [7]:
df.head(5)

,Tipo_Ataque,Exploits,Tipo_Ataque.1,Ransomware,Tipo_Ataque.2,Amenazas_web,Tipo_Ataque.3,Ataques_red,Tipo_Ataque.4,Bajo_demanda,Tipo_Ataque_Combinado
0,República de Guinea Ecuatorial,0.00365,República Islámica de Afganistán,0.00100,República de Moldova,0.02808,República de Corea,0.02492,República de Guinea-Bissau,0.03080,República de Guinea Ecuatorial
1,República de la Unión de Myanmar,0.00156,Turkmenistán,0.00085,República de Letonia,0.02663,República Socialista de Vietnam,0.01109,Turkmenistán,0.02693,República de la Unión de Myanmar
2,Estados Unidos de América,0.00128,Reino de Lesotho,0.00085,República de Malawi,0.02638,República Popular de China,0.01032,República Argelina Democrática y Popular,0.02682,Estados Unidos de América
3,Guayana,0.00126,República Islámica de Pakistán,0.00078,República de Madagascar,0.02608,República de Ghana,0.01014,República Islámica de Afganistán,0.02666,Guayana
4,Canadá,0.00107,República Islámica de Irán,0.00078,"Nación de Brunei, Morada de la Paz",0.02577,República Islámica de Irán,0.00871,República de la Unión de Myanmar,0.02625,Canadá


In [8]:


# Unir las columnas de tipo de ataque en una sola columna
df['Tipo_Ataque_Combinado'] = df['Tipo_Ataque'].fillna(df['Tipo_Ataque.1']).fillna(df['Tipo_Ataque.2']).fillna(df['Tipo_Ataque.3']).fillna(df['Tipo_Ataque.4'])

# Seleccionar las columnas numéricas
columnas_numericas = ['Exploits', 'Ransomware', 'Amenazas_web', 'Ataques_red', "Bajo_demanda"]

# Derretir el DataFrame para convertir las columnas numéricas en filas
df_melted = pd.melt(df, id_vars=['Tipo_Ataque_Combinado'], value_vars=columnas_numericas, var_name='Columna', value_name='Valor')

# Pivotar el DataFrame para obtener las columnas numéricas en columnas separadas
df_final = df_melted.pivot_table(index='Tipo_Ataque_Combinado', columns='Columna', values='Valor', aggfunc='first')

# Restaurar el índice
df_final.reset_index(inplace=True)

# Imprimir el nuevo DataFrame
df_final


Columna,Tipo_Ataque_Combinado,Amenazas_web,Ataques_red,Bajo_demanda,Exploits,Ransomware
0,Asociado de Puerto Rico,0.01410,0.00129,0.00960,0.00029,0.00005
1,Bailía de Jersey,0.02304,0.00459,0.02188,0.00080,0.00033
2,Belice,0.01386,0.00126,0.00929,0.00027,0.00005
3,Bosnia y Herzegovina,0.00930,0.00045,0.00515,0.00006,NaN
4,Burkina Faso,0.02107,0.00277,0.01833,0.00065,0.00016
...,...,...,...,...,...,...
173,Sultanato de Omán,0.01238,0.00087,0.00843,0.00020,0.00003
174,Territorio de la Isla Norfolk,0.00642,0.00016,0.00277,NaN,NaN
175,Turkmenistán,0.01499,0.00145,0.01029,0.00033,0.00005
176,Ucrania,0.01004,0.00058,0.00569,0.00011,NaN


In [9]:
# Comparar las columnas Tipo_Ataque_Combinado y las columnas numéricas en el DataFrame original
comparacion = df[['Tipo_Ataque.1'] + columnas_numericas].equals(df_final[['Tipo_Ataque_Combinado'] + columnas_numericas])

# Imprimir el resultado
print(f"Las columnas Tipo_Ataque_Combinado y las columnas numéricas coinciden: {comparacion}")

Las columnas Tipo_Ataque_Combinado y las columnas numéricas coinciden: False


In [10]:
# OBTENER TIPOS DE ATQUE COMPARANDO FILÑAS DE LOS DF ORIGINAL Y FIONAL PARA SABER LA COINCIDENCIAS Y DISCREPANCIAS Y SU PROPORCION
# Inicializar contadores
conteo_true = 0
conteo_false = 0

# Iterar sobre cada tipo de ataque y verificar si coincide en el nuevo DataFrame
for tipo_ataque in df['Tipo_Ataque'].unique():
    mascara_originales = df['Tipo_Ataque'] == tipo_ataque
    mascara_combinados = df_final['Tipo_Ataque_Combinado'] == tipo_ataque
    
    valores_originales = df.loc[mascara_originales, columnas_numericas].values
    valores_combinados = df_final.loc[mascara_combinados, columnas_numericas].values
    
    # Verificar si los valores coinciden
    coinciden = (valores_originales == valores_combinados).all()
    
    # Actualizar los contadores
    if coinciden:
        conteo_true += 1
    else:
        conteo_false += 1

    # Imprimir el resultado para cada tipo de ataque
    print(f"Los valores para '{tipo_ataque}' coinciden: {coinciden}")

# Calcular la proporción
total_comparaciones = conteo_true + conteo_false
proporcion_true = conteo_true / total_comparaciones
proporcion_false = conteo_false / total_comparaciones

# Imprimir el conteo final y la proporción
print(f"\nNúmero de coincidencias: {conteo_true}")
print(f"Número de discrepancias: {conteo_false}")
print(f"Proporción de coincidencias: {proporcion_true:.2%}")
print(f"Proporción de discrepancias: {proporcion_false:.2%}")




Los valores para 'República de Guinea Ecuatorial' coinciden: True
Los valores para 'República de la Unión de Myanmar' coinciden: True
Los valores para 'Estados Unidos de América' coinciden: True
Los valores para 'Guayana' coinciden: True
Los valores para 'Canadá' coinciden: True
Los valores para 'República de Níger' coinciden: True
Los valores para 'Principado de Andorra' coinciden: True
Los valores para 'República Federal de Alemania' coinciden: True
Los valores para 'República de Malawi' coinciden: True
Los valores para 'República Socialista de Vietnam' coinciden: True
Los valores para 'República Centroafricana' coinciden: True
Los valores para 'Gran Ducado de Luxemburgo' coinciden: True
Los valores para 'Mancomunidad de Australia' coinciden: True
Los valores para 'República Argelina Democrática y Popular' coinciden: True
Los valores para 'Serenísima República de San Marino' coinciden: True
Los valores para 'Bailía de Jersey' coinciden: True
Los valores para 'Principado de Liechtenst

In [11]:
# AL SER UN PORCETAJE MUCHO MENOR NO TENGO EN CUENTA LAS DUISCREPACIAS

In [14]:
df_final #(tiene solo 35 NaN)
df_final.rename(columns={"Tipo_Ataque_Combinado": "Otros_territorios_atacados"}, inplace=True)

In [16]:
df_final

Columna,Otros_territorios_atacados,Amenazas_web,Ataques_red,Bajo_demanda,Exploits,Ransomware
0,Asociado de Puerto Rico,0.01410,0.00129,0.00960,0.00029,0.00005
1,Bailía de Jersey,0.02304,0.00459,0.02188,0.00080,0.00033
2,Belice,0.01386,0.00126,0.00929,0.00027,0.00005
3,Bosnia y Herzegovina,0.00930,0.00045,0.00515,0.00006,NaN
4,Burkina Faso,0.02107,0.00277,0.01833,0.00065,0.00016
...,...,...,...,...,...,...
173,Sultanato de Omán,0.01238,0.00087,0.00843,0.00020,0.00003
174,Territorio de la Isla Norfolk,0.00642,0.00016,0.00277,NaN,NaN
175,Turkmenistán,0.01499,0.00145,0.01029,0.00033,0.00005
176,Ucrania,0.01004,0.00058,0.00569,0.00011,NaN


In [19]:
df_modificado.head(1)

,Fecha_hora,IP Origen,Destino IP,Puerto_origen,Puerto_destino,Protocolo,Longitud_paquete,Tipo_paquete,Tipo,Datos_carga_util,...,Firma_ataque,respuesta_tomada,Nivel_severidad,Informacion_usuario,Informacion_dispositivo,Segmento_red,Datos_geolocalizacion,Información_proxy,Registros_cortafuegos,Fuente_registro
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis iusto accusamus ad perferendis esse at. Asperiores neque at ad.\nMaiores possimus ipsum saepe vitae. Ad possimus veritatis.,...,Known Pattern B,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.2; Trident/5.0),Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,Server


In [144]:
print(len(df_modificado.columns))

26


In [20]:
df_modificado.columns

Index(['Fecha_hora', 'IP Origen', 'Destino IP', 'Puerto_origen',
       'Puerto_destino', 'Protocolo', 'Longitud_paquete', 'Tipo_paquete',
       'Tipo', 'Datos_carga_util', 'Resultado_analisis', 'Cluster',
       'Etiqueta_x', 'Indicadores_malware', 'Puntuaciones_anomalias',
       'Tipo_ataque', 'Firma_ataque', 'respuesta_tomada', 'Nivel_severidad',
       'Informacion_usuario', 'Informacion_dispositivo', 'Segmento_red',
       'Datos_geolocalizacion', 'Información_proxy', 'Registros_cortafuegos',
       'Fuente_registro'],
      dtype='object')

In [27]:
df_modificado_kaspersky = pd.concat([df_modificado, df_final], axis=1)

In [28]:
df_modificado_kaspersky.to_excel("E:\Cursos\BC_Data_Science\Repositorio\ONLINE_DS_THEBRIDGE_V\proyecto EDA\DATASETS\DATASETS MODIFICADOS\ciber_modificado_kaspersky.xlsx", index=False)

In [24]:
df_modificado_Karpesky.head(1)

,Fecha_hora,IP Origen,Destino IP,Puerto_origen,Puerto_destino,Protocolo,Longitud_paquete,Tipo_paquete,Tipo,Datos_carga_util,...,Datos_geolocalizacion,Información_proxy,Registros_cortafuegos,Fuente_registro,Otros_territorios_atacados,Amenazas_web,Ataques_red,Bajo_demanda,Exploits,Ransomware
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis iusto accusamus ad perferendis esse at. Asperiores neque at ad.\nMaiores possimus ipsum saepe vitae. Ad possimus veritatis.,...,"Jamshedpur, Sikkim",150.9.97.135,Log Data,Server,Asociado de Puerto Rico,0.0141,0.00129,0.0096,0.00029,0.00005
